In [35]:
from sklearn import linear_model, preprocessing, impute, model_selection, metrics
from scipy.stats import boxcox
import pandas as pd
import numpy as np
import random
import os
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

In [36]:
sns.set_theme()
sns.set_style("ticks")
sns.despine()

%matplotlib inline

In [37]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
seed_everything()

In [38]:
# CONFIG
DATA_DIR = "spaceship-titanic"

def filepath(filename):
    return os.path.join(DATA_DIR,filename)

In [39]:
train_df = pd.read_csv(filepath("train.csv"),index_col="PassengerId")
test_df  = pd.read_csv(filepath("test.csv"), index_col="PassengerId")

# Add PasssengerId sinze we need it for feature engineering
# PAssengerIdは学習に使用したいのでカラムとして追加する．

train_df["PassengerId"] = train_df.index
test_df["PassengerId"]  = test_df.index

In [40]:
len(train_df),len(test_df)

(8693, 4277)

In [41]:
train_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,PassengerId
PassengerId,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001_01
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002_01
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003_01
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003_02
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004_01


In [42]:
train_df["Destination"].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

### 個人的データ解釈
- PassengerId   :そのまま`[str]`
- HomePlanet    :おうちのある星(Europa,Earth,Mars,nan)`[str]`
- CryoSleep     :コールドスリープしてるかどうか`[bool]`
- Cabin         :"[A-Z]/[0-9]/[SorP]"`[str]`
- Destination   :目的地("TRAPPIST-1e","PSO J318.5-22","55 Cancri e",nan)`[str]`
- Age           :年齢`[int]`
- VIP           :VIPかどうか`[bool]`
- RoomService   :ルームサービスでつかった金額       `[int]`
- FoodCourt     :フードコートでつかった金額         `[int]`
- ShoppingMall  :ショッピングモールでつかった金額   `[int]`
- Spa           :スパで使った金額                  `[int]`
- VRDeck        :(?)  `[int]`
- Name          :名前 `[str]`
- Transported   :到着したかどうか   `[int]`

### Initial Feature Engineering
### 特徴量エンジニアリングの初期化
#### See Spaceship Titanic - Exploratory Data Analysis
> Note: All features extraced from `Cabin` will be engineered after missing values are imputed but function is created here.  
> 注: `キャビン`から引き渡されたすべての機能は，欠損値が入力された後に設計されますが，関数はここで作成されます(?)

In [43]:
# お金使ったやつまとめる
expenditure_columns = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

#### Idの分解
- PassengerIdはGroupIdとGroupSizeに分けられる

In [44]:
def from_passengerId(df:pd.DataFrame) -> pd.DataFrame:
    """PassengerIdからグループ人数の追加

    Args:
        df (pd.DataFrame): DataFrame of csv

    Returns:
        pd.DataFrame: add "GroupId" and "GroupSize"
    """
    
    split_id = df["PassengerId"].str.split("_",expand=True)
    df["GroupId"]   = split_id[0]
    df["GroupSize"] = df.groupby("GroupId")["GroupId"].transform("count")

    # Indicates whether the passenger was traveling alone or not
    # 乗客が一人か一人ではないか

    df["Alone"] = (df["GroupSize"] == 1)

    return df

In [45]:
train_df = from_passengerId(train_df)
test_df  = from_passengerId(test_df)

In [46]:
train_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,PassengerId,GroupId,GroupSize,Alone
PassengerId,,,,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001_01,0001,1,True
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002_01,0002,1,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003_01,0003,2,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003_02,0003,2,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004_01,0004,1,True


### Presence of Missing Values
### 欠損値の存在
- 関数`missing_values_features()`は列のリストを取得し，欠損値が存在するかどうかを示す新しい列を追加する．nullが無視されないときにTotalExpense(すべての支出が列の合計)が欠落しているかどうかを示す`TotalExpense_missing`という機能の追加

In [52]:
def missing_value_features(df:pd.DataFrame,columns:list,expenditure_columns:list) ->pd.DataFrame:
    """お金を使うcolumnの中でnaの値かどうか格納されたmissingカラムの追加

    Args:
        df (pd.DataFrame): DataFrame
        columns (list): お金を使うカラム
        expenditure_columns (list): 上と同じ感じ

    Returns:
        pd.DataFrame: カラムが追加されたdf
    """ 
    for column in columns:
        df[f"{column}_missing"] = df[column].isna()
    
    df["TotalExpense_missing"] = df[expenditure_columns].sum(axis=1,skipna=False).isna()
    return df

In [50]:
columns = ["RoomService", "FoodCourt", "ShoppingMall", "Cabin", "VIP"]
train_df = missing_value_features(train_df, columns, expenditure_columns)
test_df  = missing_value_features(test_df,  columns, expenditure_columns)


In [51]:
train_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,...,PassengerId,GroupId,GroupSize,Alone,RoomService_missing,FoodCourt_missing,ShoppingMall_missing,Cabin_missing,VIP_missing,TotalExpense_missing
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,...,0001_01,0001,1,True,False,False,False,False,False,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,...,0002_01,0002,1,True,False,False,False,False,False,False
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,...,0003_01,0003,2,False,False,False,False,False,False,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,...,0003_02,0003,2,False,False,False,False,False,False,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,...,0004_01,0004,1,True,False,False,False,False,False,False


# 特徴量抽出
`from_expendingture_feature`はすべての支出カラムからその合計(特徴量)を抽出する(nullは無視)

In [53]:
def from_expenditure_feantures(df:pd.DataFrame,expenditure_columns:list) -> pd.DataFrame:
    #一人ずつ合計支出カラムの追加
    df["TotalExpense"] = df[expenditure_columns].sum(axis=1)

    return df

In [54]:
train_df = from_expenditure_feantures(train_df,expenditure_columns)
test_df  = from_expenditure_feantures(test_df,expenditure_columns)

In [55]:
train_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,...,GroupId,GroupSize,Alone,RoomService_missing,FoodCourt_missing,ShoppingMall_missing,Cabin_missing,VIP_missing,TotalExpense_missing,TotalExpense
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,...,0001,1,True,False,False,False,False,False,False,0.0
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,...,0002,1,True,False,False,False,False,False,False,736.0
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,...,0003,2,False,False,False,False,False,False,False,10383.0
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,...,0003,2,False,False,False,False,False,False,False,5176.0
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,...,0004,1,True,False,False,False,False,False,False,1091.0


### cabinの分割
`from_cabin`によって`Deck`,`Num`,`Side`の三つのカラムに分割します

In [56]:
def from_cabin(df:pd.DataFrame) -> pd.DataFrame:
    df[["CabinDeck","CabinNum","CabinSide"]] = df["Cabin"].str.split("/",expand=True)

    return df

### カテゴリかる変数のna埋め
`HomePlanet`と`CryoSleep`,`Destiation`は`feature-level`のモードによって，`Cabin`はグループモードによって埋められる(?)

In [58]:
def simple_mode_replacement(df:pd.DataFrame,columns:list) -> pd.DataFrame:
    df[columns] = df[columns].fillna(df[columns].mode().iloc[0])
    
    return df

In [59]:
columns = ["HomePlanet","CryoSleep","Destination"]
train_df= simple_mode_replacement(train_df,columns)
test_df = simple_mode_replacement(test_df,columns)

In [63]:
train_df[columns].isna().any()

HomePlanet     False
CryoSleep      False
Destination    False
dtype: bool

In [69]:
# Cabinのほう
def group_mode_replacement(df:pd.DataFrame, groupby:str or list,column:str) -> pd.DataFrame:
    # Find all passengers belonging to groups where at least one member has a non-null column value
    # 少なくとも一人，nullではないカラム値を持つメンバーを検索する
    temp = df.groupby(groupby).filter(lambda x: x[column].notna().any())
    func = lambda x: x.fillna(x.mode().iloc[0]) if x.isna().any() else x
    temp[column] = temp.groupby(groupby)[column].transform(func)

    df.loc[temp.index,column] = temp[column]

    return df

In [70]:
train_df = group_mode_replacement(train_df,groupby="GroupId",column="Cabin")
test_df  = group_mode_replacement(test_df,groupby="GroupId",column="Cabin")


今はまだtrainデータに99人の乗客が,testデータに63人が`Cabin`についてnullです．  
これの解決のために`HomePlanet`と`Destination`の最頻値によって埋めます

In [78]:
train_df = group_mode_replacement(train_df,groupby=["HomePlanet","Destination"],column="Cabin")
test_df = group_mode_replacement(test_df,["HomePlanet","Destination"],"Cabin")

### Cabinの分割
Cabinの穴埋めができたので改めてCabinを分割

In [81]:
train_df = from_cabin(train_df)
test_df = from_cabin(test_df)

In [86]:
train_df["VIP"].isna().sum(),test_df["VIP"].isna().sum()

(203, 93)

### VIPに対する考察
データより以下のことが言える
- 支出が0でコールドスリープをしていない乗客はVIPではない
- 12歳以下の乗客はVIPではない
- 地球からの乗客はVIPではない
- 火星からのVIPは18歳以上で，コールドスリープをせず，`5 Cancri e`にはいかない
このことより

In [100]:
def impute_vip_for_no_spend(df:pd.DataFrame) -> pd.DataFrame:
    #"VIP"がnull & 合計支出が0 & "CryoSleep"をしていない
    df.loc[(df["VIP"].isna()) & (df["TotalExpense"] == 0.0) & (~df["CryoSleep"]),"VIP"] = False
    return df

def impute_vip_for_children(df:pd.DataFrame) -> pd.DataFrame:
    df.loc[(df["VIP"].isna()) & (df["Age"] <= 12), "VIP"] = False
    return df

def impute_vip_for_earthling(df:pd.DataFrame) -> pd.DataFrame:
    df.loc[(df["VIP"].isna()) & (df["HomePlanet"] == "Earth"), "VIP"] = False
    return df

def impute_vip_for_martians(df:pd.DataFrame) ->pd.DataFrame:
    df.loc[(df["VIP"].isna()) & (df["Age"] >= 18) & (~df["CryoSleep"]) & (df["Destination"] != "55 cancri e"),"VIP"] = True
    return df

def impute_vip(df:pd.DataFrame) -> pd.DataFrame:
    df = impute_vip_for_no_spend(df)
    df = impute_vip_for_children(df)
    df = impute_vip_for_earthling(df)
    df = impute_vip_for_martians(df)
    return df

In [101]:
train_df = impute_vip(train_df)
test_df  = impute_vip(test_df)

In [103]:
test_df["VIP"].isna().sum()

22

In [104]:
#ランダムで決める
def impute_vip_by_prob(df):
    probs = df["VIP"].value_counts() / df["VIP"].notna().sum()
    values = np.random.choice([False, True], size=df["VIP"].isna().sum(), p=probs)
    df.loc[df["VIP"].isna(), "VIP"] = values
    df["VIP"] = df["VIP"].astype(bool)
    return df

train_df = impute_vip_by_prob(train_df)
test_df = impute_vip_by_prob(test_df)

In [106]:
test_df["VIP"].isna().sum()

0

### いらない特徴量の削除
`PassengerId`,`Cabin`,`Name`は関係ないので消します.

In [107]:
drop = ["PassengerId","Cabin","Name"]
train_df = train_df.drop(drop,axis=1)
test_df  = test_df.drop(drop,axis=1)

In [108]:
# nullの確認
train_df.isna().any()

HomePlanet              False
CryoSleep               False
Destination             False
Age                      True
VIP                     False
RoomService              True
FoodCourt                True
ShoppingMall             True
Spa                      True
VRDeck                   True
Transported             False
GroupId                 False
GroupSize               False
Alone                   False
RoomService_missing     False
FoodCourt_missing       False
ShoppingMall_missing    False
Cabin_missing           False
VIP_missing             False
TotalExpense_missing    False
TotalExpense            False
CabinDeck               False
CabinNum                False
CabinSide               False
dtype: bool

### 特徴量のエンコーディング
二つのデータセットをつなげる`concat_train_test()`と分ける`split_train_test()`  
でも`CabinNum`と`GroupSize`はexperimentの一つなので無視します．

In [114]:
def concat_train_test(train:pd.DataFrame,test:pd.DataFrame,has_labels=False) -> tuple:
    transported = None
    
    #testデータにこのラベルはないのでもしもあったら削除しなきゃ
    if has_labels:
        transported = train["Transported"].copy()
        train = train.drop("Transported",axis=1)
    
    train_index = train.index
    test_index  = test.index

    df = pd.concat([train,test])

    return df,train_index,test_index,transported

def split_train_test(df,train_index,test_index,transported=None):
    if transported is not None:
        train_df["Transported"] = transported
    
    test_df = df.loc[test_index,:]

    return train_df,test_df

In [115]:
df, train_idx, test_idx, transported = concat_train_test(train_df, test_df, has_labels=True)
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,RoomService_missing,FoodCourt_missing,ShoppingMall_missing,Cabin_missing,VIP_missing,TotalExpense_missing,TotalExpense,CabinDeck,CabinNum,CabinSide
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,0.0,B,0,P
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,...,False,False,False,False,False,False,736.0,F,0,S
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,...,False,False,False,False,False,False,10383.0,A,0,S
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,...,False,False,False,False,False,False,5176.0,A,0,S
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,...,False,False,False,False,False,False,1091.0,F,1,S


### bool2int
ロジスティック回帰ではカテゴリ変数をboolのまま操作できないのでintに変換します

In [116]:
def bool2int(df):
    columns = [column for column in df.columns if df[column].dtype.name == "bool"]
    df[columns] = df[columns].astype(int)
    return df

In [117]:
df = bool2int(df)

In [118]:
df.head()
# VIPとかがbinaryになった！

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,RoomService_missing,FoodCourt_missing,ShoppingMall_missing,Cabin_missing,VIP_missing,TotalExpense_missing,TotalExpense,CabinDeck,CabinNum,CabinSide
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,Europa,0,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.0,B,0,P
0002_01,Earth,0,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,...,0,0,0,0,0,0,736.0,F,0,S
0003_01,Europa,0,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,...,0,0,0,0,0,0,10383.0,A,0,S
0003_02,Europa,0,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,...,0,0,0,0,0,0,5176.0,A,0,S
0004_01,Earth,0,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,...,0,0,0,0,0,0,1091.0,F,1,S


### CabinSideって…
CabinSideは`S`と`P`からなるのでbinaryに変換できます．

In [119]:
df["CabinSide"] = df["CabinSide"].map({"S":0,"P":1})

### カテゴリ変数をダミー変数にする
owari

In [120]:
to_be_encoded = ["HomePlanet","Destination","GroupSize","CabinDeck"]
df = pd.get_dummies(df,columns=to_be_encoded)

In [121]:
df.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,GroupId,Alone,...,GroupSize_7,GroupSize_8,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0001,1,...,0,0,0,1,0,0,0,0,0,0
0002_01,0,24.0,0,109.0,9.0,25.0,549.0,44.0,0002,1,...,0,0,0,0,0,0,0,1,0,0
0003_01,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0003,0,...,0,0,1,0,0,0,0,0,0,0
0003_02,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0003,0,...,0,0,1,0,0,0,0,0,0,0
0004_01,0,16.0,0,303.0,70.0,151.0,565.0,2.0,0004,1,...,0,0,0,0,0,0,0,1,0,0


なんか分割するらしい

In [122]:
train_df,test_df = split_train_test(df,train_idx,test_idx,transported)

In [123]:
train_df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,RoomService_missing,FoodCourt_missing,ShoppingMall_missing,Cabin_missing,VIP_missing,TotalExpense_missing,TotalExpense,CabinDeck,CabinNum,CabinSide
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,0.0,B,0,P
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,...,False,False,False,False,False,False,736.0,F,0,S
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,...,False,False,False,False,False,False,10383.0,A,0,S
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,...,False,False,False,False,False,False,5176.0,A,0,S
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,...,False,False,False,False,False,False,1091.0,F,1,S


?

In [ ]:
def impute_missing_using_knn(df,numeric_cols,has_labels=False):
    x = df
    if has_labels:
        transported = df["Transported"]
        x = df.drop("Trainsported",axis=1)

        scaler = preprocessing.StandardScaler()
        x[numeric_cols] = scaler.fit_transform(x[numeric_cols])

        impute = impute.KNNImputer(n_neighbors=5,weights="distance")
        x = impute.fit_transform(x)